### Se importan las librerías necesarias para realizar la clasificación

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers.legacy import Adam
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


In [4]:
# Parámetros
img_size = 256  # Tamaño de la imagen ajustado a 256x256
batch_size = 32
num_classes = 4
epochs = 20
learning_rate = 0.0001
data_dir = '/Users/baudi/AI/practicas/uvas/data'  # Carpeta principal de datos

In [5]:
# Directorios
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
test_dir = os.path.join(data_dir, 'test')

In [6]:
# Generadores de datos con aumentación
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

val_datagen = ImageDataGenerator(rescale=1./255)

### Generador de imágenes de entrenamiento

In [7]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 2843 images belonging to 4 classes.


### Generador de imágenes de validación

In [8]:
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 608 images belonging to 4 classes.


In [9]:
# Creación del modelo utilizando EfficientNetB0 con pesos preentrenados en ImageNet
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Reducción de dimensiones utilizando promedio global
x = Dense(128, activation='relu')(x)  # Capa densa con 128 neuronas y activación ReLU
x = Dropout(0.2)(x)  # Capa de dropout para prevenir sobreajuste
predictions = Dense(num_classes, activation='softmax')(x)  # Capa de salida con función de activación softmax
model = Model(inputs=base_model.input, outputs=predictions)

Metal device set to: Apple M2 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2023-04-02 18:19:54.215069: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-02 18:19:54.215104: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
# Congelar capas base para utilizar transfer learning
for layer in base_model.layers:
    layer.trainable = False

In [11]:
# Compilación del modelo con optimizador Adam, pérdida de entropía cruzada categórica y métrica de precisión
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])


In [12]:
# Entrenamiento y validación del modelo
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator
)

Epoch 1/20


2023-04-02 18:20:07.150706: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-04-02 18:20:09.342833: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


89/89 [==============================] - ETA: 0s - loss: 1.3257 - accuracy: 0.3204

2023-04-02 18:20:29.713280: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


89/89 [==============================] - 25s 244ms/step - loss: 1.3257 - accuracy: 0.3204 - val_loss: 1.3173 - val_accuracy: 0.3405
Epoch 2/20
89/89 [==============================] - 20s 229ms/step - loss: 1.3182 - accuracy: 0.3327 - val_loss: 1.3151 - val_accuracy: 0.3405
Epoch 3/20
89/89 [==============================] - 21s 230ms/step - loss: 1.3220 - accuracy: 0.3204 - val_loss: 1.3139 - val_accuracy: 0.3405
Epoch 4/20
89/89 [==============================] - 21s 233ms/step - loss: 1.3198 - accuracy: 0.3306 - val_loss: 1.3176 - val_accuracy: 0.3405
Epoch 5/20
89/89 [==============================] - 21s 230ms/step - loss: 1.3187 - accuracy: 0.3338 - val_loss: 1.3141 - val_accuracy: 0.3405
Epoch 6/20
89/89 [==============================] - 21s 231ms/step - loss: 1.3190 - accuracy: 0.3254 - val_loss: 1.3129 - val_accuracy: 0.3405
Epoch 7/20
89/89 [==============================] - 21s 230ms/step - loss: 1.3181 - accuracy: 0.3296 - val_loss: 1.3148 - val_accuracy: 0.3405
Epoch 8/20

In [13]:
# Evaluación en el conjunto de test
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 611 images belonging to 4 classes.


In [14]:
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')

20/20 [==============================] - 3s 147ms/step - loss: 1.3144 - accuracy: 0.3404
Test accuracy: 0.3404255509376526


In [15]:
# Matriz de confusión y métricas adicionales
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

print('Classification Report:')
print(classification_report(test_generator.classes, y_pred, target_names=test_generator.class_indices.keys()))

print('Confusion Matrix:')
print(confusion_matrix(test_generator.classes, y_pred))

2023-04-02 18:34:38.125855: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


20/20 [==============================] - 4s 140ms/step
Classification Report:
                            precision    recall  f1-score   support

         Grape___Black_rot       0.00      0.00      0.00       177
Grape___Esca_Black_Measles       0.34      1.00      0.51       208
       Grape___Leaf_blight       0.00      0.00      0.00       162
           Grape___healthy       0.00      0.00      0.00        64

                  accuracy                           0.34       611
                 macro avg       0.09      0.25      0.13       611
              weighted avg       0.12      0.34      0.17       611

Confusion Matrix:
[[  0 177   0   0]
 [  0 208   0   0]
 [  0 162   0   0]
 [  0  64   0   0]]


/Users/baudi/miniconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/baudi/miniconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/baudi/miniconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
